In [2]:
# -*- coding: utf-8 -*-
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import os
import re
import pycountry_convert as pc
from datetime import datetime as dt
#brew update
#brew install enchant
import enchant

# import customized functions for dashboard
import dash_func

##############################################################
##############################################################
#######################Dash Board#############################
##############################################################
##############################################################


# read data and parse the date column
pd_df = pd.read_pickle('random_ts.pkl')
# drop off duplications
pd_no_dup = pd_df.drop_duplicates()



# Initialization for dash
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe',
    'AT': 'Antarctica'
 }
# setup colors for later usage
colors = {
    'background': '#FFFFFF',
    'text': '#7FDBFF'
}

#########################################
######Main components of Dash#############
app.layout = html.Div(
    style = {'backgroundColor': colors['background']}, 
    children = [
        # title
    html.H1(
        children = 'Dashboard for Topline Metrics',
        style = {
            'textAlign': 'center',
            'color': colors['text']
        }
    ),
    
###########Define Filters##############
    html.H2(
        children = 'Filters',
        style = {
            'textAlign': 'left',
            'color': colors['text']
        }
    ),
    
    html.Div([
    html.Label('Platforms'),
    dcc.Checklist(
        id = 'platform',
        options = dash_func.get_attrs(pd_no_dup,'Platform'),
        value = [dash_func.get_attrs(pd_no_dup,'Platform')[0]['label']],
        style = {'width': '48%'}),
        
    html.Label('Metrics'),
    dcc.RadioItems(
        id = 'metric',
        options = dash_func.get_attrs(pd_no_dup,'Metrics'),
        value = dash_func.get_attrs(pd_no_dup,'Metrics')[0]['label'],
        labelStyle = {'display': 'inline-block'},),
    ],style = {'width': '48%', 'display': 'inline-block'}),
    
    
    html.Div([
    html.Label('Continents: NOT WORKING YET'),
    dcc.Checklist(
        id = 'continents',
        options = dash_func.get_attrs(pd_no_dup,'Continents'),
        value = [c['label'] for c in dash_func.get_attrs(pd_no_dup,'Continents')],
        style = {'width': '48%'}),
        
    html.Label('Countries'),
    dcc.Dropdown(
        id = 'country',
        options = dash_func.get_attrs(pd_no_dup,'Country'),
        value = [dash_func.get_attrs(pd_no_dup,'Country')[0]['label'],dash_func.get_attrs(pd_no_dup,'Country')[1]['label'],dash_func.get_attrs(pd_no_dup,'Country')[100]['label'],],
        multi = True,)
    ], style = {'width': '48%', 'float': 'right', 'display': 'inline-block'}),
        
############Define Visulizations#################
    html.H2(
        children='Visulizations',
        style={
            'textAlign': 'left',
            'color': colors['text']
        }
    ),
    
    
    html.Div([
            html.Div([
            html.Label('Date Range:'),
            dcc.DatePickerRange(
                id='my-date-picker-range',
                #'Date Range: 2018-03-15 00:00:00 to 2020-03-09 00:00:00, Total 724 Days'
                min_date_allowed=min(pd_no_dup.index.unique()),
                max_date_allowed=max(pd_no_dup.index.unique()),
                initial_visible_month=min(pd_no_dup.index.unique()),
                #initial_visible_month=min(pd_no_dup.index.unique()),
                start_date = min(pd_no_dup.index.unique()),
                end_date = max(pd_no_dup.index.unique())
            ),
            html.Div(id='output-container-date-picker-range'),
        ],style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label('Y-scale'),
            dcc.RadioItems(
                id='yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            ),  
            ],
        style={'width': '48%', 'display': 'inline-block'}),
    html.Label('Scatter-Line Plot'),
    dcc.Graph(
        id='scatter-line'
        )
    ]),
###############Define Tables####################    
    html.H2(
        children='Tables',
        style={
            'textAlign': 'left',
            'color': colors['text']
        }
    ),
    html.H4(children='Selected Rows'),
    html.Div(id = 'table'),
      
    
])


################Define Callbacks################
@app.callback(
    dash.dependencies.Output('output-container-date-picker-range', 'children'),
    [dash.dependencies.Input('my-date-picker-range', 'start_date'),
     dash.dependencies.Input('my-date-picker-range', 'end_date')])
def update_output(start_date, end_date):
    string_prefix = 'You have selected: '
    if start_date is not None:
        start_date = dt.strptime(start_date.split('T')[0], '%Y-%m-%d')
        start_date_string = start_date.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'Start Date: ' + start_date_string + ' | '
    if end_date is not None:
        end_date = dt.strptime(end_date.split('T')[0], '%Y-%m-%d')
        end_date_string = end_date.strftime('%B %d, %Y')
        string_prefix = string_prefix + 'End Date: ' + end_date_string
    if len(string_prefix) == len('You have selected: '):
        return 'Select a date to see it displayed here'
    else:
        return string_prefix


@app.callback(
    [Output('scatter-line', 'figure'),
     Output('table', 'children'),
    ],
    [Input('platform', 'value'),
     Input('country', 'value'),
     Input('metric','value'),
     Input('yaxis-type','value'),
     Input('my-date-picker-range', 'start_date'),
     Input('my-date-picker-range', 'end_date')])
def set_display_children(platform, country, metric, yaxis_type, start_date, end_date):
    '''
    Generate figure based on filters
    '''
    #global pd_no_dup 
    filters = [
        {'Platform': platform},{'Country': country}
    ]
    
    bool_filters, attr_filters = dash_func.get_filtered_data(pd_no_dup, filters)
    table = dash_func.generate_table(pd_no_dup, bool_filters, start_date, end_date)
    data = dash_func.generate_data_for_figure(pd_no_dup, metric, bool_filters, attr_filters)
    fig = {'data': data,
         'layout': dict(
                xaxis={'title': 'Time','range':[start_date,end_date]},
                yaxis={'title': metric,'type':'linear' if yaxis_type == 'Linear' else 'log'},
                margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                #legend={'x': 0, 'y': 0.3},
                hovermode='closest',
                #xaxis_range=['2019-07-01','2019-12-31']
                 )
          }
    return fig, table

if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Mar/2020 13:46:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2020 13:46:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2020 13:46:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2020 13:46:09] "POST /_dash-update-component HTTP/1.1" 200 -


2018-03-15 00:00:00


127.0.0.1 - - [24/Mar/2020 13:46:12] "POST /_dash-update-component HTTP/1.1" 200 -


2018-03-15 00:00:00
2018-03-15 00:00:00


...

127.0.0.1 - - [24/Mar/2020 13:48:03] "POST /_dash-update-component HTTP/1.1" 200 -


2018-03-15 00:00:00


127.0.0.1 - - [24/Mar/2020 13:48:28] "POST /_dash-update-component HTTP/1.1" 200 -


2018-03-15 00:00:00


127.0.0.1 - - [24/Mar/2020 13:48:33] "POST /_dash-update-component HTTP/1.1" 200 -
